### Tabular data exploration

- [Parking permits](https://data.somervillema.gov/City-Services/City-of-Somerville-Parking-Permits/xavb-4s9w) between January 1, 2017 and December 31, 2018 
- Registered vehicles - confidential file from Cortni

In [1]:
# import libraries
import pandas as pd

#### 1. Parking Permits

In [2]:
# read in full dataset for parking permits 
parking_permits = pd.read_csv('../data/City_of_Somerville_Parking_Permits.csv')

In [61]:
parking_permits.head()

,type_code,type_name,issued,effective,expiration,st_addr,unit_num,city,state,zip_code
0,WD,Moving Van,02/23/2017 12:00:00 AM,03/01/2017 12:00:00 AM,03/01/2017 12:00:00 AM,69 ADAMS ST,1,SOMERVILLE,MA,2145.0
1,G,Visitor,05/22/2017 12:00:00 AM,04/01/2017 12:00:00 AM,04/30/2018 12:00:00 AM,37 SEWALL ST,,SOMERVILLE,MA,2145.0
2,G,Visitor,05/22/2017 12:00:00 AM,04/01/2017 12:00:00 AM,04/30/2018 12:00:00 AM,37 SEWALL ST,,SOMERVILLE,MA,2145.0
3,G,Visitor,07/07/2017 12:00:00 AM,07/06/2017 12:00:00 AM,06/30/2018 12:00:00 AM,25 BEACON ST,5,SOMERVILLE,MA,2143.0
4,G,Visitor,07/07/2017 12:00:00 AM,07/06/2017 12:00:00 AM,06/30/2018 12:00:00 AM,25 BEACON ST,5,SOMERVILLE,MA,2143.0


In [62]:
# parking_permits.type_name.unique()

In [3]:
print('Size of parking permit data \t\t\t {}\nSize of Residential parking permit data \t {}'.format(parking_permits.shape, parking_permits[parking_permits.type_name == 'Residential'].shape))

Size of parking permit data 			 (172689, 10)
Size of Residential parking permit data 	 (32980, 10)


In [4]:
# subset data to only residential parking permits
residential_permits = parking_permits[parking_permits.type_name == 'Residential']

# clean date columns
residential_permits['issued'] = pd.to_datetime(residential_permits['issued'])
# residential_permits['effective'] = pd.to_datetime(residential_permits['effective'])
# residential_permits['expiration'] = pd.to_datetime(residential_permits['expiration'])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [5]:
res_permits_by_st_addr = residential_permits.groupby('st_addr').aggregate({'issued':len}).reset_index()
res_permits_by_st_addr.columns=['st_addr', 'residential_permits_issued']

In [6]:
res_permits_by_st_addr.head()

,st_addr,residential_permits_issued
0,1 ALDERSEY ST,1
1,1 AVON ST,3
2,1 BEACON ST,1
3,1 BELMONT SQ,3
4,1 BENTON RD,3


In [7]:
res_permits_by_st_addr.shape

(12617, 2)

**Noisy label option 1**  
Number of residential permits issued by street address.

Issues:
- clear inconsistency, such as for 1 Aldersey St, which has 3 garage doors from [google street view](https://www.google.com/maps/place/1+Aldersey+St,+Somerville,+MA+02143/@42.382985,-71.0960374,3a,75y,21.92h,88.7t/data=!3m6!1e1!3m4!1suVgqBBiLUdBI5VRy9pYyYA!2e0!7i16384!8i8192!4m5!3m4!1s0x89e370cca2b22e2d:0x5dbed58b8d9c69f9!8m2!3d42.3830618!4d-71.0958082)
- Data only available for 12,617 streets


- do we need info on type of house - number of units, singlefamily/multi etc


#### 2. Registered vehicles
From Cortni: 

> The spreadsheet contains one row per garaged vehicle in the City. Each unique license plate has an anonymized ID (e.g. COS_1). With vehicle registrations, you'll want to make sure you don't double count cars that share a license plate (e.g. Tom had car A for first half of the year, then traded it in for Car B and moved license plate to the new vehicle). In other words, organize data by # of unique license plates per property. Also, keep in mind the caveat we discussed that the addresses on this list are billing addresses, not the garaging address. So you will see some non-Somerville addresses or a car dealership that leases vehicles (excise is billed to dealer who charges lessee). There is also a PDF attached with a key for plate types. 

In [63]:
registered_vehicles = pd.read_excel('../data/COPY_Registered_Vehicles_16_17.xlsx', sheet_name='raw')
print(registered_vehicles.shape)
registered_vehicles = registered_vehicles[registered_vehicles.Year == 2017]
print(registered_vehicles.shape)

(102132, 9)
(51583, 9)


In [64]:
registered_vehicles.head()

,Address,Unit,City,State,Zip,Plate.Type,Year,PlateID,EV
0,67 CONCORD AVE,,SOMERVILLE,MA,2143,PAN,2017,COS_1,No
3,46 BOW ST,,SOMERVILLE,MA,2143,PAN,2017,COS_2,No
5,PO BOX 901098,,FORT WORTH,TX,76101,PAR,2017,COS_3,No
7,38 HIGH ST #1,,WOBURN,MA,1801,PAR,2017,COS_4,No
8,36 HUDSON STREET,APT #2,E SOMERVILLE,MA,2143,PAN,2017,COS_5,No


In [38]:
categories = ['PAN', 'PAR', 'PAS', 'PAV', 'PAY', 'SPN']
# 

In [40]:
registered_vehicles_select_cat = registered_vehicles[registered_vehicles['Plate.Type'].isin(categories)]

In [56]:
check = registered_vehicles_select_cat.groupby(['Address', 'City', 'Unit', 'PlateID']).aggregate(
    {'EV':len}).reset_index()

In [58]:
check.head()

,Address,City,Unit,PlateID,EV
0,,,,COS_10645,1
1,,,,COS_1250,1
2,,,,COS_13857,1
3,,,,COS_14061,1
4,,,,COS_15478,1
